In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tomoro-ai/combined.csv
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/model.safetensors.index.json
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/config.json
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/LICENSE
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/README.md
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/tokenizer.json
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/tokenizer_config.json
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/model-00001-of-000002.safetensors
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/.gitattributes
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/model-00002-of-000002.safetensors
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/generation_config.json
/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2/fi

In [2]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets trl
!pip install -q -U huggingface_hub einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 110.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 12.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 4.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)


In [4]:
!python -m pip install -U peft

In [5]:
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
import torch

In [6]:
df = pd.read_csv("/kaggle/input/tomoro-ai/combined.csv")

In [7]:
df.head()

,Unnamed: 0,Question,Answer,Program,Processed
0,0,what was the percentage change in the net cash...,14.1%,"subtract(206588, 181001), divide(#0, 181001)",Question: what was the percentage change in th...
1,1,what was the percentage change in net sales fr...,-32%,"subtract(5363, 7983), divide(#0, 7983)",Question: what was the percentage change in ne...
2,2,what portion of the total shares subject to ou...,70.1%,"add(2530454, 5923147), divide(5923147, #0)",Question: what portion of the total shares sub...
3,3,what was the percent of the change in the comp...,15.7%,"subtract(118, 102), divide(#0, 102)",Question: what was the percent of the change i...
4,4,what portion of total obligations are due with...,22.99%,"add(27729, 45161), divide(#0, 317105)",Question: what portion of total obligations ar...


In [8]:
df.columns

Index(['Unnamed: 0', 'Question', 'Answer', 'Program', 'Processed'], dtype='object')

In [9]:
dataset = Dataset.from_pandas(df)


In [10]:
model_name = "/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2"

In [11]:
# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)


In [12]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
).to("cuda")


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


In [14]:
# Prepare model for training
model = prepare_model_for_kbit_training(model)

In [15]:
#%% [LoRA Configuration]
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]  # For Qwen architecture
)


In [32]:
#%% [Training Arguments]
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=2,  # Reduce if OOM occurs
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    optim="paged_adamw_8bit",
    save_strategy="steps",
    save_steps=250,
    save_total_limit=1,  # Keep only the latest model, replacing the previous one
    report_to="none",
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    dataloader_pin_memory=True,  # GPU memory optimization

)


In [33]:
#%% [Initialize Trainer]
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    # dataset_text_field='Processed',
    # max_seq_length=2048,  # Reduce based on your context length
    # tokenizer=tokenizer,
    formatting_func=lambda example: example["Processed"],
    args=training_args,
    # packing=False
)


/usr/local/lib/python3.10/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/2234 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/2234 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2234 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2234 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2234 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [35]:
#%% [Modified Trainer Initialization]
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    # Updated text formatting approach
    formatting_func=lambda example: example['Processed'],  # NEW
    args=training_args,
)


/usr/local/lib/python3.10/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/2234 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/2234 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2234 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2234 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2234 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
#%% [Start Training]
trainer.train()

#%% [Save Adapter]
trainer.save_model("final_model")


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
